In [7]:
from starspot import Star as ST
from starspot import Spot as SP

In [9]:
sp1 = SP()
sp1.evolve(1e1)

[0.04290665957171763,
 0.13178337270414814,
 0.24302188391271035,
 0.3687441269455074,
 0.5053999261807469,
 0.6509492120479197,
 0.8040553018852873,
 0.9637682874265263,
 1.1293748605525211,
 1.300317603100745,
 1.4761475854366641,
 1.6564946177417934,
 1.8410475925260381,
 2.029540948685432,
 2.221745031053657,
 2.417459028561997,
 2.616505676663304,
 2.8187272012802738,
 3.023982157844633,
 3.232142929414653,
 3.443093719162712,
 3.6567289198197677,
 3.8729517747852116,
 4.091673267891308,
 4.312811194560686,
 4.536289378418835,
 4.762037005691493,
 4.989988055838929,
 5.220080811470991,
 5.452257434071559,
 5.686463594734568,
 5.922648151185525,
 5.684533438388228,
 5.4483649177395,
 5.214192207872577,
 4.982067991105497,
 4.752048322124894,
 4.524192980755791,
 4.298565877118364,
 4.075235519444825,
 3.8542755573760985,
 3.6357654168796962,
 3.419791047311104,
 3.206445806988517,
 2.9958315215587246,
 2.788059760280773,
 2.583253390474375,
 2.3815484918252143,
 2.1830967432523396,

In [12]:
st1 = ST()
st1.simulate_spots()